In [143]:
#Import libraries
%matplotlib inline
import os
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.misc import imresize
import keras

In [2]:
#Load images
phototexture    = []

folder=os.getcwd()+'/photos/'+'texture/'
texture_dict = {}

filelist = glob.glob(folder+'/*.png')
textureimages = np.array([imresize(np.array(Image.open(fname).convert('RGBA')),10) for fname in filelist])

In [3]:
texturefiles = np.array([np.array(fname.split('/')[-1].replace('.png','')) for fname in filelist])
texturefiles = list(map(int, texturefiles))

In [124]:
texture = pd.read_csv('texture.csv')
texture = texture.drop(['Unnamed: 0'],axis=1)
texture['texture'] = texture.texture.str.replace(',','.')
texture['texture'] = pd.to_numeric(texture['texture'])
texture['inx']   = texture.index

In [127]:
texturefiles = pd.DataFrame(texturefiles,columns=['inx'])
texturedata = texturefiles.merge(texture,on='inx')

In [134]:
errorimages = [1126,1132,1143]
errorindex  = texturedata[texturedata.inx.isin(errorimages)].index.values
texturedata = texturedata.drop(errorindex)

In [136]:
texturedata = texturedata.drop(['filename','empresa','gravel_size','image','inx'],axis=1)

In [165]:
#texturedata['category'] = pd.cut(texturedata.texture,bins=[-0.01,165,275,350,460,20000],labels=['Polido',
#                                             'Macio', 'Baixa Rugosidade', 'Rugosidade Moderada','Alta Rugosidade'])
texturedata['category'] = pd.cut(texturedata.texture,bins=[-0.01,165,275,350,460,20000],labels=[0, 1, 2, 3, 4])

In [138]:
textureimagesclean = np.delete(textureimages,errorindex,0)

In [166]:
x_train = textureimagesclean[:]
y_train = texturedata['category'][:]

In [167]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=1)

In [168]:
from keras.utils import np_utils

# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [31]:

# print shape of training set
print('x_train shape:', x_train.shape)

# print number of training, validation, and test images
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
#print(x_valid.shape[0], 'validation samples')


x_train shape: (1425, 48, 64, 4)
1425 train samples
425 test samples
